[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

<h1>Анализ тональности отзывов на фильмы:
строим простые модели</h1>
 

В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

Мы будем использовать стандартный датасет из nltk, уже возникавший в одном из примеров в предыдущих курсах. Для того, чтобы импортировать необходимый модуль, напишите:

 
<code>from nltk.corpus import movie_reviews</code>
 

Чтобы получить id-шники негативных и позитивных отзывов:

<code> 
negids = movie_reviews.fileids('neg')<br>
posids = movie_reviews.fileids('pos')
</code> 

Чтобы получить список негативных отзывов:

<code> 
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
</code> 

<h2>Инструкция по выполнению</h2>

В некоторых пунктах нужно получить ответ - число или строку, которые будет нужно набирать в текстовых файлах и прикреплять в ответах на вопросы. Десятичные дроби записывайте через точку.

<ol><li>Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.</li>
<li>Подсчитайте количество отзывов в выборке.</li>
<li>Подсчитайте долю класса 1 в выборке.</li>
<li>Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.</li>
<li>Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.</li>
<li>Аналогично accuracy, оцените качество по ROC AUC.
Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.</li>
</ol>

In [ ]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

Загружаем датасет

In [ ]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

Формируем массив отзывов и их меток (0 - отрицательный отзыв, 1 - положительный)

In [ ]:
negids = movie_reviews.fileids('neg') 
posids = movie_reviews.fileids('pos')
print('Количество положительных отзывов', len(posids))
print('Количество отрицательных отзывов', len(negids))

neg_reviews = [' '.join(movie_reviews.words(fileids=[f])) for f in negids]
pos_reviews = [' '.join(movie_reviews.words(fileids=[f])) for f in posids]
corpus = neg_reviews + pos_reviews
labels = [0] * len(neg_reviews) + [1] * len(pos_reviews)
print('Общее количество отзывов', len(corpus))

Количество положительных отзывов 1000
Количество отрицательных отзывов 1000
Общее количество отзывов 2000


In [ ]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(corpus)
print('Количество признаков', x.shape[1])

Количество признаков 39659


Оценим качество классификации, рассчитав accuracy и roc-auc

In [ ]:
%%time
clf = LogisticRegression(max_iter=1000)
pline = Pipeline([("vectorizer", vectorizer), ("classifier", clf)])
print('Точность классификации', cross_val_score(pline, corpus, labels).mean())

Точность классификации 0.8445
CPU times: user 20 s, sys: 16.5 s, total: 36.6 s
Wall time: 26.3 s


In [ ]:
print('roc auc', cross_val_score(pline, corpus, labels, scoring='roc_auc').mean())

roc auc 0.9162949999999999


Обучаем модель на всей выборке

In [ ]:
pline.fit(corpus, labels)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression(max_iter=1000))])

Выводим 5 наиболее значимых слов, им соответствуют максимальные по модулю веса модели

In [ ]:
imp_indexes = np.argsort(np.abs(clf.coef_[0]))[-5:][::-1]
res = [vectorizer.get_feature_names()[i] for i in imp_indexes]
''.join(res)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'badunfortunatelyworstfunnothing'